# Tutorial: Buscador Semântico de PDFs

Este notebook mostra como criar um buscador inteligente que entende o significado dos textos em arquivos PDF. Você poderá pesquisar por assuntos, não só por palavras exatas.

**Exemplo:** Se você procurar por "energia renovável", o sistema pode encontrar textos que falam sobre "painéis solares" ou "energia eólica", mesmo que não usem exatamente as mesmas palavras.

## 1. Instalação dos Pacotes

**O que está acontecendo:**
Instalamos as bibliotecas necessárias para manipular PDFs, criar representações numéricas dos textos (embeddings), dividir textos em partes menores e construir o banco de dados para buscas.

**Por quê:**
Essas bibliotecas são a base para todo o pipeline de busca semântica.

**Exemplo:**
Imagine que cada frase do PDF vira uma lista de números. Assim, o computador consegue comparar o significado dos textos.

In [5]:
# Instale as bibliotecas necessárias (execute apenas uma vez)
!pip install langchain langchain-community pypdf faiss-cpu langchain-huggingface


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Importação das Bibliotecas

**O que está acontecendo:**
Aqui importamos as ferramentas que vão ler os PDFs, dividir os textos, criar os vetores e armazenar tudo para busca.

**Por quê:**
Cada biblioteca tem uma função específica no processo.

**Exemplo:**
- `os` serve para navegar pelas pastas do computador.
- `PyPDFLoader` lê o conteúdo dos PDFs.
- `RecursiveCharacterTextSplitter` quebra textos grandes em pedaços menores.
- `HuggingFaceEmbeddings` transforma textos em vetores (listas de números).
- `FAISS` é o banco de dados rápido para buscar por significado.

In [6]:
# 2. Importação das Bibliotecas
# O que está acontecendo:
# Aqui importamos as bibliotecas essenciais para o pipeline de busca semântica.
# - os: manipulação de arquivos e diretórios
# - PyPDFLoader: carrega PDFs e transforma em documentos
# - RecursiveCharacterTextSplitter: divide textos em pedaços menores (chunks)
# - HuggingFaceEmbeddings: gera vetores semânticos dos textos
# - FAISS: banco vetorial para buscas rápidas

import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


## 3. Carregando os PDFs

**O que está acontecendo:**
O código abaixo lê todos os arquivos PDF de uma pasta e junta todas as páginas em uma lista.

**Por quê:**
Assim, conseguimos trabalhar com todos os textos dos PDFs de uma vez.

**Exemplo:**
Se você tem 3 PDFs com 10 páginas cada, o sistema vai juntar as 30 páginas em uma lista para processar.

In [7]:
# Defina o diretório onde os PDFs enviados pelo usuário estão salvos
pdf_dir = r"C:\Users\Guilh\OneDrive\Área de Trabalho\Projetos IA\TutorialChain\Docs\Artigos"   # Altere para o nome da sua pasta de uploads

# Lista para armazenar todos os documentos carregados
docs = []

# Percorre todos os arquivos da pasta e carrega os PDFs
for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(pdf_dir, filename))
        docs.extend(loader.load())

print(f"Total de arquivos PDF encontrados: {len([f for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')])}")
print(f"Total de páginas carregadas: {len(docs)}")

Total de arquivos PDF encontrados: 7
Total de páginas carregadas: 722


## 4. Dividindo o Texto em Partes Menores (Chunks)

**O que está acontecendo:**
Os textos dos PDFs são divididos em pedaços menores, chamados de "chunks".

**Por quê:**
Buscar em textos menores é mais eficiente e preciso. Assim, o sistema pode mostrar só o trecho relevante.

**Exemplo:**
Se uma página tem 3.000 caracteres, ela pode ser dividida em 3 pedaços de 1.000 caracteres cada.

In [8]:
# Divide os textos em chunks menores para facilitar a busca semântica
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

print(f"Total de chunks gerados: {len(chunks)}")

Total de chunks gerados: 2877


## 5. Criando os Vetores (Embeddings)

**O que está acontecendo:**
Cada pedaço de texto é transformado em um vetor (lista de números) que representa o significado daquele texto.

**Por quê:**
O computador não entende palavras, mas entende números. Assim, conseguimos comparar o significado dos textos.

**Exemplo:**
A frase "gato preto" pode virar o vetor `[0.12, -0.34, 0.56, ...]`.

In [9]:
# Cria o objeto de embeddings usando um modelo HuggingFace (especializado em transformar texto em vetores)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

c:\Users\Guilh\OneDrive\Área de Trabalho\Projetos IA\TutorialChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 6. Criando e Salvando o Banco Vetorial

**O que está acontecendo:**
Os vetores dos textos são armazenados em um banco de dados especial (FAISS), que permite buscas rápidas por significado.

**Por quê:**
Assim, quando você faz uma pergunta, o sistema encontra rapidamente os textos mais parecidos com o que você procurou.

**Exemplo:**
Se você perguntar "Como funciona energia solar?", o sistema busca os vetores mais próximos dessa pergunta.

In [10]:
# Antes de usar FAISS, é necessário instalar o pacote faiss-cpu (ou faiss-gpu se tiver GPU compatível).
# Execute a linha abaixo se ainda não instalou:
!pip install faiss-cpu

from langchain.vectorstores import FAISS

# Cria o banco vetorial com os embeddings dos chunks
vector_store = FAISS.from_documents(chunks, embeddings)

# Salve o banco vetorial FAISS após criar para evitar reprocessamento
vector_store.save_local("meu_banco_vetorial")  # Salva o banco vetorial em disco


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 7. Exibindo os Resultados da Busca

**O que está acontecendo:**
Aqui você pode filtrar e visualizar os resultados encontrados pelo buscador semântico.

**Por quê:**
Assim, você vê o título, página e um trecho do texto encontrado, facilitando a leitura.

**Exemplo:**
Se você buscar por "inteligência artificial", o sistema mostra o título do artigo, a página e um pedaço do texto onde o assunto aparece.

---

### Como fazer uma busca:
1. Escreva sua pergunta ou termo de interesse na variável `consulta`.
2. O sistema irá procurar os textos mais parecidos com sua pergunta.

**Exemplo de consulta:**
- "O que é aprendizado de máquina?"
- "Exemplos de energia renovável"

In [16]:
# Faça uma busca semântica no banco vetorial
consulta = "O que é inteligência artificial?"  # Altere para sua pergunta
resultados_busca = vector_store.similarity_search(consulta, k=5)  # Retorna os 5 textos mais próximos

# Salva os resultados na variável 'results' para exibição
results = resultados_busca

In [17]:
palavra_chave_titulo = ""   # Deixe vazio para mostrar todos, ou coloque uma palavra para filtrar

for i, doc in enumerate(results, 1):
    titulo = doc.metadata.get('title', '').lower()
    trecho = doc.page_content[:500].replace('\n', ' ')
    if not palavra_chave_titulo or palavra_chave_titulo in titulo:
        print(f'--- Resultado {i} ---')
        print(f'Título: {doc.metadata.get("title", "Sem título")}')
        print(f'Página: {doc.metadata.get("page", "Desconhecida")}')
        print('Trecho:')
        print(trecho)
        print('-' * 60)

--- Resultado 1 ---
Título: 
Página: 296
Trecho:
CHAPTER 14 Question Answering, Informa- tion Retrieval, and Retrieval- Augmented Generation People need to know things. So pretty much as soon as there were computers we were asking them questions. Systems in the 1960s were answering questions about baseball statistics and scientiﬁc facts. Even ﬁctional computers in the 1970s like Deep Thought, invented by Douglas Adams inThe Hitchhiker’s Guide to the Galaxy, answered “the Ultimate Question Of Life, The Universe, and Everything”. 1 And because s
------------------------------------------------------------
--- Resultado 2 ---
Título: 
Página: 335
Trecho:
(Levesque et al., 1990), and shared plans (Grosz and Sidner, 1980). The earliest conversational systems were simple pattern-action chatbots like ELIZA (Weizenbaum, 1966). ELIZA had a widespread inﬂuence on popular perceptions of artiﬁcial intelligence, and brought up some of the ﬁrst ethical questions in natural language processing —such 